In [1]:
!pip install alpaca_trade_api

     |████████████████████████████████| 1.3 MB 36.6 MB/s 
     |████████████████████████████████| 52 kB 950 kB/s 
     |████████████████████████████████| 636 kB 33.6 MB/s 
     |████████████████████████████████| 103 kB 46.0 MB/s 
     |████████████████████████████████| 160 kB 38.3 MB/s 
     |████████████████████████████████| 271 kB 55.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install alpaca_backtrader_api

     |████████████████████████████████| 410 kB 42.7 MB/s 
     |████████████████████████████████| 40 kB 5.9 MB/s 
     |████████████████████████████████| 108 kB 58.0 MB/s 
  Created wheel for trading-calendars: filename=trading_calendars-2.1.1-py3-none-any.whl size=140937 sha256=9f53711c52d9e0f3b52caff19cd728d6a19f17c831201e72de9efa52e0d66168
  Stored in directory: /root/.cache/pip/wheels/62/9c/d1/46a21e1b99e064cba79b85e9f95e6a208ac5ba4c29ae5962ec
Successfully built trading-calendars
  Attempting uninstall: alpaca-trade-api
    Found existing installation: alpaca-trade-api 1.4.1
    Uninstalling alpaca-trade-api-1.4.1:
      Successfully uninstalled alpaca-trade-api-1.4.1


**Our strategy is that if the 1-minute average is above the 20-minute average, we are going to buy it. Otherwise, clear the stock. This strategy will run every minute.**

In [12]:
API_KEY = "PK8SSSXD3FX164IYREJ5"
SECRET_KEY = "NPKEgtQyt3fsRodbOqn4d46V19XI5W9S8Cv64dDT"

In [13]:
import datetime
import time

import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd

pd.options.display.max_rows = 999
pd.set_option('display.max_columns', None)

from datetime import timedelta

from pytz import timezone

tz = timezone('EST')

api = tradeapi.REST(API_KEY,
                    SECRET_KEY,
                    'https://paper-api.alpaca.markets')

import logging

# setting the length of moving average and checking the frequency
slow = 20
fast = 1

freq = '1Min'



Here is the ticker we want to trade and the number of purchases

In [ ]:
#select stock and BUY NUMBER FOR EACH PURCHASE #
loading = {'AA': 300, 'AAL': 300, 'UAL': 300, 'NIO': 250, 'AMD': 300, 'NCLH': 200, 'BYND': 200, 'DAL': 500, 'ATVI': 300,
        'WORK': 200, 'VIRT': 200, 'AAPL': 300, 'AMC': 200, 'TSLA': 80, 'NKLA': 180, 'XPEV': 100, 'NVDA': 50,
           'LMT': 80, 'ZM': 80, 'DOCU': 100, 'TWLO': 100, 'CRWD': 100, 'EAR': 80, 'SNOW': 80, 'TWTR': 400, 'EA': 250,
           'ABBV': 200, 'CRSR': 1000, 'PFE': 500, 'PDD': 200, 'LI': 200, 'DISCK':80 }

# create a list to hold only stock symbols #
symbols = list(loading)

Then, we are ready to write our strategy.

In [ ]:
def get_data_bars(symbols, rate, slow, fast):
    # generate log to keep track of every transaction
    logging.basicConfig(filename='log/{}.log'.format(time.strftime("%Y%m%d")))
    logging.warning('{} logging started'.format(datetime.datetime.now().strftime("%x %X")))

    # combine the price per minute and volume of all stocks into a dataframe 

    # use API to generate the dataframe including the final prices and volume
    try:
        data = api.get_barset(symbols, rate, limit=20).df
 
        # the data that has been accquired
        ticker = [x.symbol for x in api.list_positions()]  # the stock ticker 
        qty = [x.qty for x in api.list_positions()]        # positions
        avg_entry_price = [x.avg_entry_price for x in api.list_positions()]
        my_position = dict(zip(ticker, qty))
        entry_price = dict(zip(ticker, avg_entry_price))

        # "fast_ema_1min" is average price per minute
        # "loading" is the number of shares
        # "PL" is profit and loss 
        for x in (list(loading)):

            data.loc[:, (x, 'fast_ema_1min')] = data[x]['close'].rolling(window=fast).mean()
            data.loc[:, (x, 'slow_ema_20min')] = data[x]['close'].rolling(window=slow).mean()
            data.loc[:, (x, 'return_1_min')] = (data[x]['close']- data[x]['close'].shift(1))/(data[x]['close'].shift(1))
            data.loc[:, (x, 'diff')] = data[x]['slow_ema_20min'] - data[x]['fast_ema_1min']
            data.loc[:, (x, 'loading')] = int(loading[x])

            if x in ticker:
                data.loc[:, (x, 'qty')] = int(my_position[x])
                data.loc[:, (x, 'entry_price')] = float(entry_price[x])
            else:
                data.loc[:, (x, 'qty')] = 0
                data.loc[:, (x, 'entry_price')] = data[x]['close']
            data.loc[:, (x, 'PL')] = (data[x]['close'] - data[x]['entry_price']) * (data[x]['qty'])

        data.fillna(method='ffill', inplace=True)
        return data
    except:
        print("There might be connection errors")
        pass

Next, we check all selected stocks against the data in the dataframe and generate **trading signals**.

In [ ]:
def get_signal_bars(symbol_list, rate, ema_slow, ema_fast):
    now = datetime.datetime.now()
    data = get_data_bars(symbol_list, rate, ema_slow, ema_fast)

    signals = {}
    for x in symbol_list:    
        # When the one-minute average price exceeds the 20-minute average price,
        # Buy into!
        if (data[x].iloc[-1]['fast_ema_1min'] >= data[x].iloc[-1]['slow_ema_20min']):
            signal = (data[x].iloc[-1]['loading'])

        # When the one-minute average price falls below the 20-minute average price,
        # Clear the stock!
        else:
            signal = (data[x].iloc[-1]['qty'])*(-1)
        signals[x] = signal

    return signals

U.S. markets open Monday through Friday from 9:30 a.m. to 4 p.m. Est. The function below adjusts the running time of the code.

In [ ]:
def time_to_open(current_time):
    if current_time.weekday() <= 4:
        d = (current_time + timedelta(days=1)).date()
    else:
        days_to_mon = 0 - current_time.weekday() + 7
        d = (current_time + timedelta(days=days_to_mon)).date()
    next_day = datetime.datetime.combine(d, datetime.time(8, 30, tzinfo=tz))
    seconds = (next_day - current_time).total_seconds()
    return seconds

We are done. Let's run the program.

In [ ]:
def run_checker(stocklist):
    print('run_checker started')
    while True:
        # Check if today is a trading day(Monday - Friday)
        if datetime.datetime.now(tz).weekday() >= 0 and datetime.datetime.now(tz).weekday() <= 4:
            # Checks if market is open now
            print('Trading in process '+ datetime.datetime.now().strftime("%x %X"))
            if datetime.datetime.now(tz).time() > datetime.time(8,30) and datetime.datetime.now(tz).time() <= datetime.time(15, 00):
                signals = get_signal_bars(stocklist, freq, slow, fast)
                for signal in signals:
                    if signals[signal] > 0:
                        
                        try:
                            api.submit_order(signal, signals[signal], 'buy', 'market', 'day')
                            print('{} bought {}  {} shares, portfolio value {}   '.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal], api.get_account().equity))
                            # Record purchases of stocks
                            logging.warning('{} bought {}  {} shares, portfolio value {}   '.format(datetime.datetime.now(tz).strftime("%x %X"),
                                                                                                            signal, signals[signal], api.get_account().equity))
                        except:
                            # Record if underfunding
                            logging.warning('{} Insufficient buying power'.format(datetime.datetime.now(tz).strftime("%x %X")))
                            print('Trading in process '+ datetime.datetime.now().strftime("%x %X") + ' buying ' + signal + ' but Insufficient fund')
                            pass
                    elif signals[signal] < 0:
                        try:
                            api.submit_order(signal, -signals[signal], 'sell', 'market', 'day')
                            print('{} sold   {} {} shares, portfolio value {}   '.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal], api.get_account().equity))
                            # Record sales of stocks
                            logging.warning('{} sold   {} {} shares, portfolio value {}   '.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal], api.get_account().equity))
                        except Exception as e:
                            pass
                           # Rest for 60 seconds before regenerating the signal
                time.sleep(60)

            else:
                # The program stops running until the market opens 
                # Get time amount until open, sleep that amount
                print('Market closed ({})'.format(datetime.datetime.now(tz)))
                print('Sleeping', round(time_to_open(datetime.datetime.now(tz))/60/60, 2), 'hours')
                time.sleep(time_to_open(datetime.datetime.now(tz)))

In [ ]:
if __name__ == "__main__":
    run_checker(symbols)

run_checker started
Trading in process 11/16/21 00:03:05
Market closed (2021-11-15 19:03:05.049131-05:00)
Sleeping 13.45 hours


The key columns for the Alpaca API's native dataframe are price and volume. However, we extracted the key information from the object and incorporated it into the dataframe. For example, we have merged the qty and avg_entry_price of each stock into the dataframe to generate signals.